# Faster Optimizers

To make training faster we can also use faster optimizer than the regular Gradient Descent.

## Momentum Optimization

Gradient Descent updates the weights by directly subtracting the gradient of the cost function multiplied learning rate, it does not care about what the previous gradients were.

Momentum optimization cares a lot about the previous gradients: at each iteration, it subtracts the local gradient from the *momentum vector* $m$, and it updates the weights by adding this new momentum vector. There is a new hyperparameter $\beta$, called *momentum*. It controls speed of momentum and prevent momentum from growing too large, which must be set between 0 and 1. A typical value is 0.9.
$$
m \gets \beta m - \eta \nabla_\theta J(\theta) \\
\theta \gets \theta + m
$$
The maximum size of the weight updates is equal to that gradient multiplied by the learning rate multiplied by $\frac1{1 - \beta}$. This way if $\beta=0.9$ then optimization ends up 10 times faster than Gradient Descent! This allows momentum optimization to escape from plateaus and local optima much faster.

> Due to the momentum, the optimizer may overshoot a bit, then comes back, overshoot again, and oscillate like this many times before stabilizing at the minimum.

This simple momentum in Keras can be implemented very easily:

```python
optimizer = keras.optimizers.SGD(lr=0.0001, momentum=0.9)
```

## Nestrov Accelerated Gradient

One small variant of momentum optimization is *Nestrov Accelerated Gradient (NAG)*. It measures gradients not at the local position $\theta$, but slightly ahead in the direction of the momentum, at $\theta + \beta m$.
$$
m \gets \beta m - \eta\nabla_\theta J(\theta + \beta m)\\
\theta \gets \theta + m
$$
Every step it is a bit closed to the optimum, it adds up and NAG converges much faster.

To use it simply set `nestrov=True` when creating the `SGD` optimizer:

```python
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nestrov=True)
```

## AdaGrad

When there is a problem of elongated bowl, GD sometimes go in wrong direction (like when input scales are different) results in slower convergence. AdaGrad solves this problem by decaying learning rate or scaling down the gradient vector when there is a steep dimension. This is called *adaptive learning rate*.
$$
s \gets s + \nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta)\\
\theta \gets \theta - \eta\nabla_\theta J(\theta) \oslash \sqrt{s + \epsilon}
$$
So it squares up the gradient vector and then square root them in updating. By this when there is a steep square values became very large resulting vector in the direction of optimum.

It is good for simple quadratic problems, but it stops earlier when training neural network as it scales down learning rate very much in that case. So, you should not use AdaGrad for neural networks. But its useful to learn the concept of adaptive learning rate.

## RMSProp

RMSProp fixes the problem of AdaGrad by accumulating just few previous gradients (rather than from the beginning). It does so by using exponential decay in the first step:
$$
s \gets \beta_s + (1 - \beta) \nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta) 
\\
\theta \gets \theta - \eta \nabla_\theta J(\theta) \oslash \sqrt{s + \epsilon}
$$
The decay rate $\beta$ is typically set to 0.9. In Keras you can use RMSProp like this:

```python
optimizer = keras.optimizers.RMSProp(lr=0.001, rho=0.9)
```

RMSProp works in every case as it is default in most research before Adam arrives.

## Adam and Nadam

Adam stands for adaptive moment estimation, combines the idea of momentum and RMSProp optimization: just like momentum optimization, it keeps track of an exponential decaying average of past gradients and squares of past gradients as RMSProp:
$$
m \gets \beta_1m - (1 - \beta_1) \nabla_\theta J(\theta)
\\
s \gets \beta_2s + (1 - \beta_2) \nabla_\theta J(\theta) \otimes \nabla_\theta J(\theta)
\\
\hat{m} \gets \frac{m}{1-\beta_1^t}
\\
\hat{s} \gets \frac{s}{1-\beta_2^t}
\\
\theta \gets \theta + \eta \hat{m} \oslash\sqrt{\hat{s} + \epsilon}
$$
Here, $t$ indicates the iteration number. 

We have to change value of $m$ and $s$, because they are initialized to zero, they are biased to zero. So, those two steps boosts $m$ and $s$ at the beginning of the training.

The momentum decay parameter $\beta_1$ is set to 0.9, while scaling decay hyperparameter $\beta_2$ is often initialized to 0.999. 

Here is how to use Adam in Keras:

```python
optmizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```

Since Adam is an adaptive learning rate algorithm, it requires less tuning of learning rate. You can start by $\eta=0.001$, making Adam even easier to use Gradient Descent.

Finally, two variants of Adam:

- *Ada Max*:
  - In this it changes only second step to $s \gets \max(\beta_2s, \nabla_\theta J(\theta))$. This can make Adam more stable, but it really depends on the dataset. In general Adam performs well.
- Nadam:
  - It is Adam plus the Nestrov trick, so it will converge slightly faster than Adam.

> For some dataset adaptive optimizations are just not generalize well, so you can use simple Nestrov Accelerated Gradients technique when it happens.

